<a href="https://colab.research.google.com/github/codemail-sudhanshu/Cataract-Dataset/blob/main/dataAugumentationCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q keras

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, 
from tensorflow.keras.utils import img_to_array , array_to_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications import ResNet50


import matplotlib.pyplot as plt

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Deep Learning Models")
!ls

mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5
sq_model.h5
SQUEEZENET.docx
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#Augumentation with only zoom and grayscale format

train_datagen2 = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip = True,
                             zoom_range = 0.3,
                             rescale=1/255.0
                            )

#Corresponding generator
train_generator2 = train_datagen2.flow_from_directory(
    train,
    color_mode='grayscale',
    target_size = (128,128),
    batch_size=32,
    class_mode = 'categorical',
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen2 = ImageDataGenerator(
     rescale=1/255.0 )
#Using testing data as validation data

validation_generator2 = test_datagen2.flow_from_directory(
    test,
    color_mode='grayscale',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42 )

x_train,y_train = next(train_generator2)
x_test,y_test = next(validation_generator2)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 1)
(32, 128, 128, 1)


In [ ]:
#Basic CNN Model
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
import numpy as np
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint_filepath = 'checkpoint/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu', input_shape = (128, 128, 1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
#for fitting the model
from keras.callbacks import EarlyStopping

stop = EarlyStopping(
    monitor = 'val_accuracy',
    mode='max',
    patience= 8,
    verbose = 2,
    restore_best_weights = True
)

STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS,
callbacks=stop
)

Epoch 1/20
12/12 [==============================] - 94s 9s/step - loss: 140.9354 - accuracy: 0.4630 - val_loss: 7.8532 - val_accuracy: 0.5469
Epoch 2/20
12/12 [==============================] - 2s 153ms/step - loss: 3.4570 - accuracy: 0.5238 - val_loss: 0.9715 - val_accuracy: 0.5938
Epoch 3/20
12/12 [==============================] - 2s 130ms/step - loss: 0.9277 - accuracy: 0.5212 - val_loss: 0.9670 - val_accuracy: 0.3750
Epoch 4/20
12/12 [==============================] - 2s 132ms/step - loss: 0.8635 - accuracy: 0.5000 - val_loss: 0.9583 - val_accuracy: 0.3750
Epoch 5/20
12/12 [==============================] - 2s 132ms/step - loss: 0.8818 - accuracy: 0.5529 - val_loss: 0.8771 - val_accuracy: 0.4531
Epoch 6/20
12/12 [==============================] - 2s 133ms/step - loss: 0.8592 - accuracy: 0.5212 - val_loss: 0.9018 - val_accuracy: 0.4062
Epoch 7/20
12/12 [==============================] - 2s 193ms/step - loss: 0.8862 - accuracy: 0.5661 - val_loss: 1.1221 - val_accuracy: 0.3281
Epoch 

**Starting With ResNet50**

In [ ]:
from keras import applications
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

from keras.optimizers import SGD, Adam

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights= None, include_top=False, input_shape= (128,128,1))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation= 'softmax')(x)
resnetmodel = Model(inputs = base_model.input, outputs = predictions)
adam = Adam(lr=0.001)
resnetmodel.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
EPOCH = 30
stopping_resNet = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5,
    verbose = 2,
    restore_best_weights = True
)

restNetModel = resnetmodel.fit(
    train_generator2,
    steps_per_epoch=STEPS,
    epochs=EPOCH,
    validation_data=validation_generator2,
    validation_steps=VAL_STEPS,
)

Epoch 1/30
4/4 [==============================] - 12s 739ms/step - loss: 1.2465 - accuracy: 0.8241
Epoch 2/30
4/4 [==============================] - 1s 150ms/step - loss: 4.8554e-06 - accuracy: 1.0000
Epoch 3/30
4/4 [==============================] - 1s 119ms/step - loss: 2.2076e-09 - accuracy: 1.0000
Epoch 4/30
4/4 [==============================] - 1s 122ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/30
4/4 [==============================] - 1s 127ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/30
4/4 [==============================] - 1s 128ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/30
4/4 [==============================] - 1s 123ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/30
4/4 [==============================] - 1s 120ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/30
4/4 [==============================] - 1s 121ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/30
4/4 [==============================] - 1s 122ms/step - loss: 0.0000e+00 -

Simple CNN with **1 Conv Layer**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization

import numpy as np

In [ ]:
model = Sequential()

#1-conv layer
model.add(Conv2D(64, (3,3), padding = 'same', input_shape=(128, 128,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#Flatten
model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(3,activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      640       
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 128, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 262144)           

In [ ]:
STEPS = train_generator2.samples//train_generator2.batch_size
VAL_STEPS = validation_generator2.samples//validation_generator2.batch_size
EPOCHS = 20

results = model.fit(
train_generator2,
steps_per_epoch=STEPS,
epochs=EPOCHS,
validation_data=validation_generator2,
validation_steps=VAL_STEPS
)

Epoch 1/20
12/12 [==============================] - 5s 155ms/step - loss: 1.2224 - accuracy: 0.5079 - val_loss: 1.5984 - val_accuracy: 0.3125
Epoch 2/20
12/12 [==============================] - 2s 134ms/step - loss: 0.7569 - accuracy: 0.7037 - val_loss: 0.8941 - val_accuracy: 0.6172
Epoch 3/20
12/12 [==============================] - 2s 137ms/step - loss: 0.6278 - accuracy: 0.7434 - val_loss: 0.9242 - val_accuracy: 0.7422
Epoch 4/20
12/12 [==============================] - 2s 139ms/step - loss: 0.5488 - accuracy: 0.7672 - val_loss: 0.9452 - val_accuracy: 0.5703
Epoch 5/20
12/12 [==============================] - 2s 137ms/step - loss: 0.5141 - accuracy: 0.7910 - val_loss: 1.0693 - val_accuracy: 0.4297
Epoch 6/20
12/12 [==============================] - 2s 165ms/step - loss: 0.4234 - accuracy: 0.8439 - val_loss: 1.1676 - val_accuracy: 0.4062
Epoch 7/20
12/12 [==============================] - 2s 179ms/step - loss: 0.3612 - accuracy: 0.8677 - val_loss: 1.3606 - val_accuracy: 0.4062
Epoch 

In [ ]:
model2 = Sequential()

#1-conv layer
model2.add(Conv2D(64, (3,3), padding = 'same', input_shape=(256, 256,1)))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))


#2-conv layer
model2.add(Conv2D(128,(5,5),padding='same'))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size = (2,2)))
model2.add(Dropout(0.25))



#Flatten
model2.add(Flatten())


model2.add(Dense(256))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))

model2.add(Dense(512))
model2.add(BatchNormalization())
model2.add(Activation('relu'))
model2.add(Dropout(0.25))


#output layer
model2.add(Dense(3,activation='softmax'))

opt = Adam(lr = 0.0001)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])

model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 256, 256, 64)      640       
                                                                 
 batch_normalization_2 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 256, 256, 64)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 128, 128, 64)      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 128, 128, 128)    

In [ ]:
results_3layer = model2.fit(
    x=train_generator2,
    steps_per_epoch = STEPS,
    epochs = EPOCHS,
    validation_data = validation_generator2,
    validation_steps = VAL_STEPS,
)

Epoch 1/20


InvalidArgumentError: ignored

**VGG16**

In [ ]:
#For RGB images

train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             rescale=1/255.0,
                            )

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
     rescale=1/255.0 )
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    target_size = (128,128),
    class_mode = 'categorical',
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)


In [ ]:
from keras.applications.vgg16 import VGG16
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping

vgg16_weight_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model=tf.keras.applications.vgg16.VGG16(
    # include_top=False,
    weights='imagenet',
    include_top=False, 
    input_shape=(128,128,3,)
)
# base_model = VGG16(
#     weights=vgg16_weight_path,
#     include_top=False, 
#     input_shape=(128,128,3,)
# )

NUM_CLASSES = 3

model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

model.layers[0].trainable = False

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_8 (Dropout)         (None, 8192)              0         
                                                                 
 dense_9 (Dense)             (None, 3)                 24579     
                                                                 
Total params: 14,739,267
Trainable params: 24,579
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
EPOCHS_VGG = 30
STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

VGG_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=10,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_VGG,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[VGG_STOP]
)

Epoch 1/30
12/12 [==============================] - 8s 441ms/step - loss: 1.0514 - accuracy: 0.5053 - val_loss: 0.8264 - val_accuracy: 0.5703
Epoch 2/30
12/12 [==============================] - 4s 297ms/step - loss: 0.6773 - accuracy: 0.6958 - val_loss: 0.7676 - val_accuracy: 0.6250
Epoch 3/30
12/12 [==============================] - 3s 250ms/step - loss: 0.5286 - accuracy: 0.7831 - val_loss: 0.5962 - val_accuracy: 0.7344
Epoch 4/30
12/12 [==============================] - 3s 242ms/step - loss: 0.4191 - accuracy: 0.8360 - val_loss: 0.5688 - val_accuracy: 0.7969
Epoch 5/30
12/12 [==============================] - 4s 317ms/step - loss: 0.3466 - accuracy: 0.8783 - val_loss: 0.5568 - val_accuracy: 0.8047
Epoch 6/30
12/12 [==============================] - 3s 244ms/step - loss: 0.3070 - accuracy: 0.8968 - val_loss: 0.5905 - val_accuracy: 0.7109
Epoch 7/30
12/12 [==============================] - 3s 221ms/step - loss: 0.2857 - accuracy: 0.9074 - val_loss: 0.6201 - val_accuracy: 0.6875
Epoch 

Mobile Net **V2**

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from keras import layers
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop

In [ ]:
#Training and testing directories
train = 'TRAIN_DIR/'
test = 'TEST_DIR/'

#For RGB images
train_datagen = ImageDataGenerator(horizontal_flip=True, 
                             zoom_range = 0.3,
                             preprocessing_function = preprocess_input)

#Corresponding generator
train_generator = train_datagen.flow_from_directory(
    train,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42)

#ImageDataGenerator for testing/validation data

test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input
)
#Using testing data as validation data

validation_generator = test_datagen.flow_from_directory(
    test,
    color_mode='rgb',
    batch_size=32,
    class_mode = 'categorical',
    target_size = (128,128),
    seed=42 )

x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

Found 410 images belonging to 3 classes.
Found 140 images belonging to 3 classes.
(32, 128, 128, 3)
(32, 128, 128, 3)


In [ ]:
mobilenet_weight_path = 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'

mobileNet = tf.keras.applications.MobileNetV2(  input_shape = (128,128,3),alpha=1.0,include_top=False,weights='imagenet')
# mobileNet = MobileNetV2(
#     input_shape = (128,128,3),
#     alpha=1.0,
#     include_top=False,
#     weights= mobilenet_weight_path
# )

MN_model = Sequential()
MN_model.add(mobileNet)
MN_model.add(layers.GlobalAveragePooling2D())
MN_model.add(layers.Dropout(0.5))
MN_model.add(layers.Dense(3, activation='softmax'))
    
MN_model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=0.00002),
        metrics=['accuracy']
    )

MN_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_2   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_9 (Dropout)         (None, 1280)              0         
                                                                 
 dense_10 (Dense)            (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 2,227,715
Non-trainable params: 34,112
_________________________________________________________________


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#for fitting the model
from keras.callbacks import EarlyStopping

EPOCHS_MN = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

MN_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = MN_model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_MN,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[MN_STOP]
)

Epoch 1/30
12/12 [==============================] - 12s 467ms/step - loss: 1.5280 - accuracy: 0.3360 - val_loss: 1.1590 - val_accuracy: 0.4297
Epoch 2/30
12/12 [==============================] - 3s 281ms/step - loss: 1.2058 - accuracy: 0.4709 - val_loss: 0.9906 - val_accuracy: 0.5000
Epoch 3/30
12/12 [==============================] - 3s 255ms/step - loss: 0.9514 - accuracy: 0.5608 - val_loss: 0.8914 - val_accuracy: 0.5469
Epoch 4/30
12/12 [==============================] - 4s 314ms/step - loss: 0.7221 - accuracy: 0.6878 - val_loss: 0.8706 - val_accuracy: 0.5469
Epoch 5/30
12/12 [==============================] - 3s 237ms/step - loss: 0.6295 - accuracy: 0.7370 - val_loss: 0.7932 - val_accuracy: 0.6172
Epoch 6/30
12/12 [==============================] - 3s 265ms/step - loss: 0.4675 - accuracy: 0.8360 - val_loss: 0.7013 - val_accuracy: 0.7188
Epoch 7/30
12/12 [==============================] - 4s 306ms/step - loss: 0.4025 - accuracy: 0.8598 - val_loss: 0.7519 - val_accuracy: 0.7109
Epoch

**SQUEEZENET**

In [56]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.12.0


In [57]:
bnmomemtum=0.9
def fire(x, squeeze, expand):
  y  = tf.keras.layers.Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
  y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
  y1 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
  y1 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y1)
  y3 = tf.keras.layers.Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
  y3 = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y3)
  return tf.keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
  return lambda x: fire(x, squeeze, expand)

x = tf.keras.layers.Input(shape=[128,128, 3]) # input is 192x192 pixels RGB

y = tf.keras.layers.Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
y = tf.keras.layers.BatchNormalization(momentum=bnmomemtum)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(64, 128)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(48, 96)(y)
y = tf.keras.layers.MaxPooling2D(pool_size=2)(y)
y = fire_module(24, 48)(y)
y = tf.keras.layers.GlobalAveragePooling2D()(y)
y = tf.keras.layers.Dense(3, activation='softmax')(y)

model = tf.keras.Model(x, y)
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 128, 128, 32  896         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 128, 128, 32  128        ['conv2d_4[0][0]']               
 rmalization)                   )                                                           

In [58]:
x_train,y_train = next(train_generator)
x_test,y_test = next(validation_generator)
print(x_train.shape)
print(x_test.shape)

(32, 128, 128, 3)
(32, 128, 128, 3)


In [59]:
#for fitting the model
from keras.callbacks import EarlyStopping
EPOCHS_SQ = 30

STEPS = train_generator.samples//train_generator.batch_size
VAL_STEPS = validation_generator.samples//validation_generator.batch_size

SQ_STOP = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=7,
    verbose = 2,
    restore_best_weights = True
)

history = model.fit(
    train_generator,
    steps_per_epoch=STEPS,
    epochs=EPOCHS_SQ,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS,
    callbacks=[SQ_STOP]
)

Epoch 1/30
12/12 [==============================] - 25s 837ms/step - loss: 0.8742 - accuracy: 0.6349 - val_loss: 1.6432 - val_accuracy: 0.3203
Epoch 2/30
12/12 [==============================] - 3s 230ms/step - loss: 0.4998 - accuracy: 0.8776 - val_loss: 2.2807 - val_accuracy: 0.2969
Epoch 3/30
12/12 [==============================] - 5s 400ms/step - loss: 0.3670 - accuracy: 0.8968 - val_loss: 1.2868 - val_accuracy: 0.4062
Epoch 4/30
12/12 [==============================] - 3s 273ms/step - loss: 0.2598 - accuracy: 0.9524 - val_loss: 0.6583 - val_accuracy: 0.7500
Epoch 5/30
12/12 [==============================] - 3s 256ms/step - loss: 0.2012 - accuracy: 0.9630 - val_loss: 0.4048 - val_accuracy: 0.8750
Epoch 6/30
12/12 [==============================] - 3s 228ms/step - loss: 0.1360 - accuracy: 0.9762 - val_loss: 0.4120 - val_accuracy: 0.8750
Epoch 7/30
12/12 [==============================] - 6s 483ms/step - loss: 0.1104 - accuracy: 0.9868 - val_loss: 0.4281 - val_accuracy: 0.8359
Epoch

In [62]:
model.save('sq_model.h5')

ValueError: ignored